In [ ]:
import nltk
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
from textblob import TextBlob
import requests
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('words')
stop_words = stopwords.words('english')
words = set(nltk.corpus.words.words())
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
  def load_data_label(path):
      temp_data = []
      lines = [line.strip() for line in open(path)]
      for idx in range(0, len(lines), 4):
          id = lines[idx].split("\t")[0]
          relation = lines[idx + 1]

          sentence = lines[idx].split("\t")[1][1:-1]
          sentence = sentence.replace("<e1>", " _e1_ ").replace("</e1>", " _/e1_ ")
          sentence = sentence.replace("<e2>", " _e2_ ").replace("</e2>", " _/e2_ ")
          sentence = sentence.replace("<e1>", "<e1> ").replace("</e1>", " </e11>")
          sentence = sentence.replace("<e2>", "<e2> ").replace("</e2>", " </e22>")

          tokens = nltk.word_tokenize(sentence)

          tokens.remove('_/e1_')
          tokens.remove('_/e2_')

          e1 = tokens.index("_e1_")
          del tokens[e1]
          element1=tokens[e1]
          e2 = tokens.index("_e2_")
          del tokens[e2]
          element2=tokens[e2]
          sentence = " ".join(tokens)
          temp_data.append([id, sentence, e1, element1, e2, element2, relation])
      df = pd.DataFrame(data=temp_data, columns=["id", "sentence", "e1_position","element1", "e2_position","element2", "class"])
      #print (df)
      labelsMapping = {'Other': 0,'Message-Topic(e1,e2)': 1, 'Message-Topic(e2,e1)': 2,
                      'Product-Producer(e1,e2)': 3, 'Product-Producer(e2,e1)': 4,
                      'Instrument-Agency(e1,e2)': 5, 'Instrument-Agency(e2,e1)': 6,
                      'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8,
                      'Cause-Effect(e1,e2)': 9, 'Cause-Effect(e2,e1)': 10,
                      'Component-Whole(e1,e2)': 11, 'Component-Whole(e2,e1)': 12,
                      'Entity-Origin(e1,e2)': 13, 'Entity-Origin(e2,e1)': 14,
                      'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16,
                      'Content-Container(e1,e2)': 17, 'Content-Container(e2,e1)': 18}
      df['tag'] = [labelsMapping[r] for r in df['class']]
      #print(df)
      x_sentence = df['sentence'].tolist()

      #Label Data
      y = df['tag']
      return df

In [ ]:
path_to_train_file = "/content/TRAIN_FILE.TXT"

path_to_test_file = "/content/TEST_FILE.txt"
df_test = load_data_label(path_to_test_file)
df = load_data_label(path_to_train_file)

In [ ]:
def preprocessor(sentence):

    sentence = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sentence)
    sentence = sentence.lower()
    sentence = re.sub(r'\d+','', sentence)
    sentence = sentence.replace("user", "")
    return  sentence

In [ ]:
def clean_text(df):

    train_cleaned = df['sentence'].apply(preprocessor)
    df['sentence'] = train_cleaned.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

    return df

In [ ]:
#cleaning the text
df['sentence'] = df['sentence'] + " " + df['element1'] + " " + df['element2']
df_test['sentence'] = df_test['sentence'] + " " + df_test['element1'] + " " + df_test['element2']
train_cleaned = clean_text(df)
test_cleaned = clean_text(df_test)

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
pip install transformers

In [ ]:
from transformers import BertTokenizer

# loading BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#  tokenizing set of texts
def Input_embeddings(data,length):

    input_ids = []
    attention_masks = []

    for sentence in data:

        encoded_sent = tokenizer.encode_plus(
            text=sentence,
            add_special_tokens=True,
            max_length=length,
            pad_to_max_length=True,
            return_attention_mask=True
            )
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
sentences = np.concatenate([train_cleaned.sentence.values, test_cleaned.sentence.values])
sentences_encoded = [tokenizer.encode(sent, add_special_tokens=True, truncation=True) for sent in sentences]
max_length = max([len(sent) for sent in sentences_encoded])
print('Max length: ', max_length)

Max length:  99


In [ ]:
data = [train_cleaned.sentence[0]]
token_ids = list(Input_embeddings(data,max_length)[0].squeeze().numpy())
print('Original: ', train_cleaned.sentence[0])
print('Token IDs: ', token_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  the system described above has its greatest application arrayed configuration antenna elements configuration elements
Token IDs:  [101, 1996, 2291, 2649, 2682, 2038, 2049, 4602, 4646, 9140, 2098, 9563, 13438, 3787, 9563, 3787, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime

# Split the data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_cleaned['sentence'].values,
    train_cleaned['tag'].values,
    test_size=0.1,
    random_state=42
)

# Tokenize all of the sentences and map the tokens to their word IDs.
train_input_ids, train_attention_masks = Input_embeddings(train_sentences, max_length)
val_input_ids, val_attention_masks = Input_embeddings(val_sentences, max_length)

# Convert the labels to tensors.
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create the DataLoader for our training set.
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

# Create the DataLoader for our validation set.
val_data = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)



In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",  # Use the 12-layer BERT model, with an uncased vocab.
    num_labels=19,  # The number of output labels--19 for our task.
    output_attentions=False,  # Whether the model returns attentions weights.
    output_hidden_states=False,  # Whether the model returns all hidden-states.
)

# Move the model to the device (GPU or CPU)
model.to(device)

# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8,
)

epochs = 4

total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps,
)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Function to calculate the F1 score
from sklearn.metrics import f1_score

def flat_f1(preds, labels):
    # Flatten the predictions and labels
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    # Exclude the 'Other' class (label 0) and calculate F1 for the remaining 18 classes
    f1 = f1_score(labels_flat, pred_flat, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], average='macro')
    return f1

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training loop
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        # Move batch tensors to the device
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()
    eval_accuracy, eval_f1 = 0, 0

    for batch in val_dataloader:
        # Move batch tensors to the device
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs[0]

        # Move logits and labels to CPU for metric calculation
        logits = logits.detach().cpu().numpy()
        labels = b_labels.to('cpu').numpy()

        # Calculate accuracy and F1 score
        eval_accuracy += flat_accuracy(logits, labels)
        eval_f1 += flat_f1(logits, labels)

    print(f"Validation Accuracy: {eval_accuracy / len(val_dataloader)}")
    print(f"Validation F1 Score: {eval_f1 / len(val_dataloader)}")
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 2.33
  Training epoch took: 0:01:46

Running Validation...
Validation Accuracy: 0.495
Validation F1 Score: 0.24835561968895306
  Validation took: 0:00:04

======== Epoch 2 / 4 ========
Training...

  Average training loss: 1.44
  Training epoch took: 0:01:53

Running Validation...
Validation Accuracy: 0.63125
Validation F1 Score: 0.3720536377203043
  Validation took: 0:00:04

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.98
  Training epoch took: 0:01:53

Running Validation...
Validation Accuracy: 0.70875
Validation F1 Score: 0.45870897004230327
  Validation took: 0:00:04

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.77
  Training epoch took: 0:01:52

Running Validation...
Validation Accuracy: 0.7225
Validation F1 Score: 0.47187718454385125
  Validation took: 0:00:04

Training complete!


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

# Tokenize the test sentences and prepare the test DataLoader
test_input_ids, test_attention_masks = Input_embeddings(test_cleaned['sentence'].values, max_length)
test_labels = torch.tensor(test_cleaned['tag'].values)

# Create the TensorDataset
test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# Create the DataLoader
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=32)

In [ ]:
# Test evaluation loop
model.eval()
test_accuracy, test_f1 = 0, 0
nb_test_steps = 0

for batch in test_dataloader:
    # Move batch to GPU
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]

    # Calculate predictions on GPU
    preds = torch.argmax(logits, dim=1)

    # Calculate accuracy on GPU
    correct_predictions = torch.sum(preds == b_labels).item()
    total_predictions = len(b_labels)
    tmp_test_accuracy = correct_predictions / total_predictions

    # Calculate F1 score on GPU (move to CPU temporarily for sklearn)
    tmp_test_f1 = f1_score(
        b_labels.cpu().numpy(),  # Move labels to CPU for sklearn
        preds.cpu().numpy(),     # Move predictions to CPU for sklearn
        average='macro',         # Macro-averaged F1 score
        labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]  # Exclude 'Other' class
    )

    # Accumulate accuracy and F1 score
    test_accuracy += tmp_test_accuracy
    test_f1 += tmp_test_f1
    nb_test_steps += 1

# Report final test metrics
print(f"Test Accuracy: {test_accuracy / nb_test_steps}")
print(f"Test F1 Score (excluding 'Other' class): {test_f1 / nb_test_steps}")

Test Accuracy: 0.7315289046653144
Test F1 Score (excluding 'Other' class): 0.4678686709078868
